In [ ]:
import pandas as pd
import walnut

# Example 3

### Deep neural network using multiple linear layers

The goal of this model is to classify iris species based on numerical features.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/uciml/iris and place it into the *data* directory.

In [ ]:
data_orig = pd.read_csv('data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

The labels are categorical values. To be used in the model, all data needs to be numerical. The function `categorical_to_numeric()` can be used to one-hot-encode all categorical data of a Pandas DataFrame object.

In [ ]:
data_enc = walnut.preprocessing.encoding.pd_one_hot_encode(data, columns=['Species'])
data_enc.sample(10)

Next the data is split into a training, validation and a testing dataset using the `split_train_test_val_data()` to evaluate the model later on. Before splitting the data is also shuffled, since sometimes raw data is sorted in some way.

In [ ]:
tensor = walnut.pd_to_tensor(data_enc)
t_train, t_val, t_test = walnut.preprocessing.split_train_val_test(tensor)
t_train[:5]

Features and labels are now seperated.

In [ ]:
x_train, y_train = walnut.preprocessing.split_features_labels(t_train, num_x_cols=4)
x_val, y_val = walnut.preprocessing.split_features_labels(t_val, num_x_cols=4)
x_test, y_test = walnut.preprocessing.split_features_labels(t_test, num_x_cols=4)

Neural networks tend to run into problems if values are very high. Therefore it is common to normalize the data. This can be done using the `normalize()` function, which applies min-max feature scaling to a tensor.<br><br>
$ X'=a+\frac{(X-X_{min})\cdot(b-a)}{X_{max}-X_{min}} $<br><br>, where<br><br>$ a $ ... lower bound<br>$ b $ ... upper bound

In [ ]:
x_train = walnut.preprocessing.normalize(x_train, axis=0)
x_val = walnut.preprocessing.normalize(x_val, axis=0)
x_test = walnut.preprocessing.normalize(x_test, axis=0)
x_train[:5]

### Step 2: Build the neural network structure
Here the individual layers of the neural network models are defined. For linear layers, activation functions and weight initialization methods can be defined. 

In [ ]:
import walnut.nn as nn

# init = "uniform"
# init = "normal"
# init = "xavier_uniform"
# init = "xavier_normal"
# init = "kaiming_uniform"
init = "kaiming_normal"

model = nn.Sequential([
    nn.layers.Linear(32, input_shape=(4,), act="tanh", init=init),
    nn.layers.Linear(32, act="tanh", init=init),
    nn.layers.Linear(32, act="tanh", init=init),
    nn.layers.Linear(3, act="softmax", init=init)
])

The network is compiled to internally connect it's layers and initialize the model. The SGD optimizer provides an optional momentum term and nesterov momentum.

In [ ]:
model.compile(
    optimizer=nn.optimizers.SGD(),
    loss_fn=nn.losses.Crossentropy(),
    metric=nn.metrics.Accuracy()
)

In [ ]:
model

### Step 3: Train the model

In [ ]:
train_loss_hist, val_loss_hist = model.train(x_train, y_train, epochs=200, val_data=(x_val, y_val), reset_params=False)

### Step 4: Evaluate the model
Using the defined metric, the model's performance can be evaluated using testing/validation data.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')

In [ ]:
predictions = model(x_test)
nn.analysis.plot_confusion_matrix(predictions, y_test, (3, 3), cmap='Blues')

### Step 5: Analyze the model
Usind different plots, the models performance and training behaviour can be analyzed (eg. checking for overfitting)

In [ ]:
n = 10
traces = {
    "train_loss" : [l if i < n else sum(train_loss_hist[i-(n-1):i+1])/n for i,l in enumerate(train_loss_hist)],
    "val_loss" : val_loss_hist
}
nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="loss history", x_label="epoch", y_label="loss")

If the `random` weight initialization method is used, the **tanh** activations get saturated very fast and the gradients "die out". If other initializers, such as `kaiming_normal` are used, this couteracts this behaviour. Furthermore the initial loss is lower and the model is therefore not wasting time correcting unnecessary high weight values in the beginning (Analysis inspired by Andrej Karpathy - highly recommend checking out his videos on YouTube)

In [ ]:
activations = {f"{i + 1} {l.__class__.__name__}" : l.y.data.copy() for i, l in enumerate(model.modules) if l.__class__.__name__ == "Tanh"}
nn.analysis.plot_distrbution(activations, figsize=(15, 3), title="activation distribution", bins=200)

As mentioned, the gradient of saturated neurons get very close to zero. If that happens for all batches, then the neuron is not learning and it is considererd dead (white pixels in the plot). By using the Kaiming He initialization method this can be reduced.

In [ ]:
saturations = {f"{i + 1} {l.__class__.__name__}" : (l.y.abs() > 0.99).data for i, l in enumerate(model.layers) if l.__class__.__name__ == "Tanh"}
nn.analysis.plot_images(saturations, (150, 30), "gray", plot_axis=True)

In [ ]:
weight_gradients = {f"{i + 1} {l.__class__.__name__}" : l.w.grad.copy() for i, l in enumerate(model.layers) if l.__class__.__name__ == "Linear"}
nn.analysis.plot_distrbution(weight_gradients, figsize=(15, 3), title=" weight gradient distribution", bins=200)

In [ ]:
act_gradients = {f"{i + 1} {l.__class__.__name__}" : l.y.grad.copy() for i, l in enumerate(model.layers) if l.__class__.__name__ == "Tanh"}
nn.analysis.plot_distrbution(act_gradients, figsize=(15, 3), title="activation gradient distribution", bins=200)